In [4]:
import csv
from pathlib import Path
import pandas as pd
import glob



In [5]:
headerline = ['loanIdentifier','origChannel','sellerName','origIntRate','origUPB','origLoanTerm',\
         'origDate','firstPmtDate','origLTV','origCLTV','numBorrowers','origDebtIncRatio',\
         'borrCreditScore','firstTHBI','loanPurp','propType','numUnits','occType','propState',\
         'zipCode','pMIperct','prodType','coborrCreditScore','mortInsType','relocMortInd']

In [6]:
for in_path in Path('data/Acquisition/staging').glob('*.txt'):
     out_path = in_path.with_suffix('.csv')
     print(f'in path: {in_path}')
     with in_path.open('r') as fin, out_path.open('w', newline='') as fout:
        reader = csv.DictReader(fin, fieldnames=headerline,delimiter="|")
        #print(f'reader fieldnames: {reader.fieldnames}')
        writer = csv.DictWriter(fout, reader.fieldnames, delimiter=",")
        
        writer.writeheader()
        writer.writerows(reader)

in path: data\Acquisition\staging\Acquisition_2019Q1.txt


In [7]:
# path = r'D:\Acquisition_All' # use your path
# all_files = glob.glob(path + "/*.csv")

li = []

for in_path in Path('data/Acquisition/staging').glob('*.csv'):
    df = pd.read_csv(in_path, index_col=None, header=0)
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)

In [9]:
df.shape

(297452, 25)

In [10]:
df.head()

,loanIdentifier,origChannel,sellerName,origIntRate,origUPB,origLoanTerm,origDate,firstPmtDate,origLTV,origCLTV,...,propType,numUnits,occType,propState,zipCode,pMIperct,prodType,coborrCreditScore,mortInsType,relocMortInd
0,100000913397,C,"JPMORGAN CHASE BANK, NATIONAL ASSOCIATION",5.875,324000,360,09/2018,11/2018,80,80,...,PU,1,P,CA,925,NaN,FRM,665.0,NaN,N
1,100017539727,B,OTHER,4.750,307000,360,12/2018,02/2019,90,90,...,PU,1,P,TX,770,25.0,FRM,NaN,1.0,N
2,100018053040,R,"JPMORGAN CHASE BANK, NATIONAL ASSOCIATION",4.875,256000,360,11/2018,01/2019,90,90,...,SF,1,S,NC,286,25.0,FRM,738.0,1.0,N
3,100019764317,C,"WELLS FARGO BANK, N.A.",4.875,248000,360,12/2018,02/2019,90,90,...,SF,1,P,IL,600,25.0,FRM,791.0,1.0,N
4,100019765730,B,"UNITED SHORE FINANCIAL SERVICES, LLC DBA UNITE...",4.250,490000,360,03/2019,05/2019,67,67,...,CO,1,P,CA,945,NaN,FRM,NaN,NaN,N


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297452 entries, 0 to 297451
Data columns (total 25 columns):
loanIdentifier       297452 non-null int64
origChannel          297452 non-null object
sellerName           297452 non-null object
origIntRate          297452 non-null float64
origUPB              297452 non-null int64
origLoanTerm         297452 non-null int64
origDate             297452 non-null object
firstPmtDate         297452 non-null object
origLTV              297452 non-null int64
origCLTV             297452 non-null int64
numBorrowers         297452 non-null int64
origDebtIncRatio     297356 non-null float64
borrCreditScore      297067 non-null float64
firstTHBI            297452 non-null object
loanPurp             297452 non-null object
propType             297452 non-null object
numUnits             297452 non-null int64
occType              297452 non-null object
propState            297452 non-null object
zipCode              297452 non-null int64
pMIperct      